# 1. Load the JSON data

In [257]:
import json

import pandas as pd

In [258]:
with open('../data/multicity_historical.json', 'r') as file:
    data = json.load(file)

data.keys()

dict_keys(['London', 'Singapore', 'Cairo', 'Buenos Aires', 'Mumbai'])

Let's make an empty list to hold dataframes for each city.

In [259]:
city_dfs = []

Since we have 5 cities, we can use the loop function to stucture it into dataframes.

In [260]:
for city, values in data.items():
    city_df = pd.DataFrame(values)
    city_df['city'] = city  # Add a column for the city name
    city_dfs.append(city_df)


Let's concatenate all the city dataframes into a single dataframe.

In [ ]:
df = pd.concat(city_dfs, ignore_index=True)

Display the first few rows to check the structure.

I've used the display() function I searched on Copilot that displays the output in the same table format as W03 Lab NB02 notebooks. 

In [262]:
display(df.head())

,time,precipitation_sum,precipitation_hours,city
0,2023-01-01,4.0,12.0,London
1,2023-01-02,0.2,2.0,London
2,2023-01-03,3.2,14.0,London
3,2023-01-04,0.9,5.0,London
4,2023-01-05,0.1,1.0,London


Display the last few rows to check the structure.

In [263]:
display(df.tail())

,time,precipitation_sum,precipitation_hours,city
1820,2023-12-27,0.0,0.0,Mumbai
1821,2023-12-28,0.0,0.0,Mumbai
1822,2023-12-29,0.0,0.0,Mumbai
1823,2023-12-30,0.0,0.0,Mumbai
1824,2023-12-31,0.0,0.0,Mumbai


I also want to check that there should be a total of 1825 entries (365 days x 5 cities).

In [264]:
entries = len(df)
print(entries)

1825


# 2. Calculate Key Metrics for Raininess Analysis

Now that we have obtained the relevant data and sorted it into organised tables, we can calculate the following required metrics. 

We will be using groupby() function to obtain the data from the coreresponding columns required.

- Total Rainfall: Sum of precipitation_sum for each city.

- Number of Rainy Days: Days with precipitation_sum > 0

- Average Rain Intensity: Total Rainfall / Number of Rainy Days

- Average Rain Duration: Total precipitation_hours / Number of Rainy Days



## 2.1 Total Rainfall

In [265]:
# Group by 'city' and sum the 'precipitation' for each group
total_rainfall = df.groupby('city')['precipitation_sum'].sum()

# First column is the city name, second column is the sum of precipitation
total_rainfall.columns = ['City', 'Total Rainfall']

# Display the result
display(total_rainfall)


city
Buenos Aires     916.3
Cairo             28.3
London           780.7
Mumbai          2048.1
Singapore       2364.7
Name: precipitation_sum, dtype: float64

Above, we can see there's no column header for total_rainfall. I want to add the column header, and based on Copilot, I can use reset_index()

In [266]:
# Group by 'city' and sum the 'precipitation' for each group
total_rainfall = df.groupby('city')['precipitation_sum'].sum().reset_index()

# Rename the columns for clarity where first column is the city name, second column is the sum of precipitation
total_rainfall.columns = ['City', 'Total Rainfall']

# Display the result
display(total_rainfall)


,City,Total Rainfall
0,Buenos Aires,916.3
1,Cairo,28.3
2,London,780.7
3,Mumbai,2048.1
4,Singapore,2364.7


## 2.2 Number of Rainy Days

We introduce the size() function to count the number of occurences in each group. In this case, we want to count the number of days precipitation_sum is greater than 0.

In [267]:
# Filter the DataFrame to include only rows where precipitation_sum > 0
rainy_days_df = df[df['precipitation_sum'] > 0]

# Group by 'city' and count the number of days for each city
rainy_days = rainy_days_df.groupby('city').size().reset_index()

# Rename the columns for clarity
rainy_days.columns = ['City', 'Rainy Days']

# Display the result
display(rainy_days)

,City,Rainy Days
0,Buenos Aires,150
1,Cairo,29
2,London,228
3,Mumbai,157
4,Singapore,350


## 2.3 Average Rain Intensity

Now, we can use the variable we have formulated in 3.1 and 3.2 to calculate the average rain intensity for each city.

In [268]:
# Merge the total_rainfall and rainy_days DataFrames on 'City'
rain_data = pd.merge(total_rainfall, rainy_days, on='City')

# Calculate the average rain intensity
rain_data['average_rain_intensity'] = rain_data['Total Rainfall'] / rain_data['Rainy Days']

# Rename the columns for clarity
rain_data.columns = ['City', 'Total Rainfall', 'Rainy Days', 'Average Rain Intensity']

# Display the result
display(rain_data)


,City,Total Rainfall,Rainy Days,Average Rain Intensity
0,Buenos Aires,916.3,150,6.108667
1,Cairo,28.3,29,0.975862
2,London,780.7,228,3.424123
3,Mumbai,2048.1,157,13.045223
4,Singapore,2364.7,350,6.756286


Neat! Now we have 3/4 metrics represented in the table.

## 2.4 Average Rain Duration

We have to first calculate the total rain duration for each city. Then, calculate the average rain duration per rainy day for each city.

In [269]:
# Calculate Total Rain Duration for each city
total_rain_duration = df.groupby('city')['precipitation_hours'].sum().reset_index()
total_rain_duration.columns = ['City', 'Total Rain Duration']

# Merge the total_rain_duration with the rain_data DataFrame
rain_data = pd.merge(rain_data, total_rain_duration, on='City')

# Calculate Average Rain Duration per Rainy Day for each city
rain_data['Average Rain Duration'] = rain_data['Total Rain Duration'] / rain_data['Rainy Days']

# Display the result
display(rain_data[['City', 'Average Rain Duration']])

,City,Average Rain Duration
0,Buenos Aires,5.700000
1,Cairo,2.827586
2,London,6.741228
3,Mumbai,14.050955
4,Singapore,8.522857


## 2.5 Combining all metrics together

Now that we have all our relevant metrics, let's put them all together in one table to help our analysis.

In [270]:
# Merge the total_rainfall, rainy_days, and total_rain_duration DataFrames on 'City'
rain_data = pd.merge(total_rainfall, rainy_days, on='City')
rain_data = pd.merge(rain_data, total_rain_duration, on='City')

# Calculate the Average Rain Intensity and Average Rain Duration per Rainy Day for each city
rain_data['Average Rain Intensity'] = rain_data['Total Rainfall'] / rain_data['Rainy Days']
rain_data['Average Rain Duration'] = rain_data['Total Rain Duration'] / rain_data['Rainy Days']

# Display the final DataFrame with the specified columns
display(rain_data[['City', 'Total Rainfall', 'Rainy Days', 'Average Rain Intensity', 'Average Rain Duration']])


,City,Total Rainfall,Rainy Days,Average Rain Intensity,Average Rain Duration
0,Buenos Aires,916.3,150,6.108667,5.700000
1,Cairo,28.3,29,0.975862,2.827586
2,London,780.7,228,3.424123,6.741228
3,Mumbai,2048.1,157,13.045223,14.050955
4,Singapore,2364.7,350,6.756286,8.522857
